In [1]:
from manim import *

Manim Community v0.17.3

In [343]:
%%manim -qm -v WARNING Frisbee
import numpy as np

class Frisbee(ThreeDScene):
    def construct(self):
    
        phi, theta, gamma = 0,0,0
        def update_angle(phi_=0, theta_=0, gamma_=0):
            return [phi_*DEGREES, theta_*DEGREES, gamma_*DEGREES]
            
        # Create Frisbee Surface
        ellipse_coefs = [3,3,.3]
        a=ellipse_coefs[0]
        h = ellipse_coefs[2]
        
        ellipse = Surface(
            lambda u, v: np.array([
                ellipse_coefs[0] * np.cos(u) * np.cos(v),
                ellipse_coefs[1] * np.cos(u) * np.sin(v),
                ellipse_coefs[2]*np.sin(u)
            ]), v_range=[0, 2*np.pi], u_range=[0, 2*PI],
            checkerboard_colors=[PURPLE, PURPLE],
            fill_opacity = .5,
            stroke_color = BLACK,
            resolution=(15, 32),
            stroke_width = 1,
        )
            

        # Set Camera
        phi,theta,gamma = update_angle(90,0,0)
        self.set_camera_orientation(phi=90 * DEGREES)
        
        self.add(ellipse.set_z_index(1))
        
        # Rotate frisbee
        phi,theta,gamma = update_angle(75,0,10)
        self.move_camera(phi=75 * DEGREES, gamma = 10*DEGREES)

        # Add vectors
        vel_vec = Vector(direction = [1,0,0]).next_to(ellipse, RIGHT).scale(1.3)
        vel_text = MathTex(r"\vec{v}").next_to(vel_vec, .5*UP)
        vel = Group(vel_vec, vel_text)

        # Helper func for fading velocity in/out
        def vel_in(wait_time = .5):
            self.add_fixed_in_frame_mobjects(vel_vec, vel_text)
            self.play(FadeIn(vel))
            self.wait(wait_time)
        
        def vel_out(wait_time = .5):
            self.add_fixed_in_frame_mobjects(vel_vec, vel_text)
            self.play(FadeOut(vel))
            self.wait(wait_time)
        
        vel_in()
        
        # Simulate Wind
        # Due to nature of stream, shift to the right by x_trans and give an ellipse buffer along x,y of buff_x, buff_y
        phi=75*DEGREES
        gamma = 10*DEGREES
        
        x_trans = .5
        buff_x, buff_y = .45, .15
        x_ell = a + buff_x
        y_ell = (a*np.cos(phi)+h*np.sin(phi))+buff_y
        
        
        def flow_func(pos): 
            def spheroid_projection_mask(x,y):
                A = gamma
                horiz_comp = (((x-x_trans)*np.cos(A) + y*np.cos(A))**2)/(x_ell**2)
                vert_comp = (((x-x_trans)*np.sin(A) - y*np.cos(A))**2)/(y_ell**2)
                return (horiz_comp + vert_comp) < 1 
            if spheroid_projection_mask(pos[0],pos[1]):
                return 0
            else:
                return 2*LEFT+.5*np.cos(pos[0]*100)*UP

        stream = StreamLines(flow_func, stroke_width=1, 
                             max_anchors_per_line=10, color=BLUE, virtual_time=1)
        self.add(stream)

        self.add_fixed_in_frame_mobjects(stream)
        
        stream.start_animation(warm_up=True, flow_speed=.75, time_width=1.0)
        self.wait(stream.virtual_time / stream.flow_speed)
        self.play(stream.end_animation())



        # Add CM, CP
        cm_p = Dot(point=[0,0,0], color=BLACK, radius=.1)
        cm_label = MathTex(r"\text{CM}").next_to(cm_p, UP*.5).scale(.75)
        cm = Group(cm_p, cm_label)
        
        cp_p = Dot(point=[0,0.8,h+.5], color=BLACK, radius=.1)
        cp_label = MathTex(r"\text{CP}").next_to(cp_p, UP*.5).scale(.75)
        cp = Group(cp_p, cp_label)
        cp.set_z_index(2)
        cm.set_z_index(2)
        
        
        # Rotate frisbee to show cm, cp
        phi,theta,gamma = update_angle(0,0,0)
        self.move_camera(phi=phi, theta=theta,gamma=gamma)
        self.wait(.5)
        self.add(cm, cp)
        # Fade in CM, CP
        self.play(FadeIn(cp,cm))
        self.wait(2)
        self.play(FadeOut(cp,cm))

        # rotate back
        phi,theta,gamma = update_angle(75,0,10)
        self.move_camera(phi=phi,gamma=gamma,theta=theta)
        self.wait(2)

       # mg vector
        mg_vec = Vector(direction=[0,-2.5]).shift(UP*.4).set_z_index(2)
        mg_label = MathTex(r"\text{mg}").next_to(mg_vec, DOWN).set_z_index(2)
        self.add_fixed_in_frame_mobjects(mg_vec, mg_label)

        lift_vec = Vector(direction=[0,2]).shift(UP*.7, RIGHT*1).set_z_index(2)
        lift_label = MathTex(r"\text{Lift}").next_to(lift_vec, RIGHT).set_z_index(2)

        drag_vec = Vector(direction=[-1,0]).shift(UP*.7, RIGHT*1).set_z_index(2)
        drag_label = MathTex(r"\text{Drag}").next_to(drag_vec, DOWN).set_z_index(2)

        
        
        self.add_fixed_in_frame_mobjects(lift_vec, lift_label, drag_vec, drag_label)
        self.play(FadeIn(lift_vec, lift_label, drag_vec, drag_label, mg_vec, mg_label))
        self.wait(1.5)
        
        L_vector = Arrow(start=[0,0,0],end=[5,0,0], color=BLUE, )
        L_label = MathTex("L")
        
        self.add_fixed_orientation_mobjects(L_label)
        self.move_camera(phi=75*DEGREES,theta=0,gamma=30*DEGREES, added_anims=[GrowArrow(L_vector),
                                                                               L_label.animate.next_to(L_vector, OUT)])
        
        self.move_camera(phi=75*DEGREES,theta=0,gamma=10*DEGREES, added_anims=[ScaleInPlace(L_vector, 0),
                                                                              L_label.animate.scale(.1)])
        self.remove(L_vector)
        self.wait(1.3)
        
        L_vector = Arrow(start=[0,0,0],end=[0,0,3.7], color=BLUE)
        self.begin_ambient_camera_rotation(about="theta", rate = 5)
        self.play(GrowArrow(L_vector), L_label.scale(1).animate.next_to(L_vector, OUT))
        self.wait(3)
        self.play(FadeOut(lift_vec, lift_label, drag_vec, drag_label, mg_vec, mg_label))
        self.begin_ambient_camera_rotation(about="phi", rate=-.3)
        self.wait(4)
        self.stop_ambient_camera_rotation()


Animation 4: AnimationGroup(Group):   3%|▏       | 2/72 [00:00<00:15,  4.58it/s]pipe:: Immediate exit requested
Error writing trailer of /Users/geebthomp/Desktop/Python_Projects/Manim/Frisbee/media/videos/Frisbee/480p15/partial_movie_files/Frisbee/2338071153_524239978_3021312846.mp4: Immediate exit requested
Error closing file /Users/geebthomp/Desktop/Python_Projects/Manim/Frisbee/media/videos/Frisbee/480p15/partial_movie_files/Frisbee/2338071153_524239978_3021312846.mp4: Immediate exit requested


In [342]:
%%manim -ql -v WARNING Two
import numpy as np

class Two(ThreeDScene):
    def construct(self):
        phi, theta, gamma = 0,0,0
        def update_angle(phi_=0, theta_=0, gamma_=0):
            return [phi_*DEGREES, theta_*DEGREES, gamma_*DEGREES]
        phi, theta, gamma = update_angle(75,0,10) 
        # Create Frisbee Surface
        ellipse_coefs = [3,3,.3]
        a=ellipse_coefs[0]
        h = ellipse_coefs[2]
        
        ellipse = Surface(
            lambda u, v: np.array([
                ellipse_coefs[0] * np.cos(u) * np.cos(v),
                ellipse_coefs[1] * np.cos(u) * np.sin(v),
                ellipse_coefs[2]*np.sin(u)
            ]), v_range=[0, 2*np.pi], u_range=[0, 2*PI],
            checkerboard_colors=[PURPLE, PURPLE],
            fill_opacity = .5,
            stroke_color = BLACK,
            resolution=(15, 32),
            stroke_width = 1,
        )
        self.set_camera_orientation(phi=phi,theta=theta,gamma=gamma)
        self.add(ellipse.set_z_index(1))
         # Add vectors
        vel_vec = Vector(direction = [1,0,0]).next_to(ellipse, RIGHT).scale(1.3)
        vel_text = MathTex(r"\vec{v}").next_to(vel_vec, .5*UP)
        vel = Group(vel_vec, vel_text)
        
        self.add_fixed_in_frame_mobjects(vel_vec, vel_text)
        self.play(FadeIn(vel))

        mg_vec = Vector(direction=[0,-2.5]).shift(UP*.4).set_z_index(2)
        mg_label = MathTex(r"\text{mg}").next_to(mg_vec, DOWN).set_z_index(2)
        self.add_fixed_in_frame_mobjects(mg_vec, mg_label)

        lift_vec = Vector(direction=[0,2]).shift(UP*.7, RIGHT*1).set_z_index(2)
        lift_label = MathTex(r"\text{Lift}").next_to(lift_vec, RIGHT).set_z_index(2)

        drag_vec = Vector(direction=[-1,0]).shift(UP*.7, RIGHT*1).set_z_index(2)
        drag_label = MathTex(r"\text{Drag}").next_to(drag_vec, DOWN).set_z_index(2)

        
        
        self.add_fixed_in_frame_mobjects(lift_vec, lift_label, drag_vec, drag_label)
        self.play(FadeIn(lift_vec, lift_label, drag_vec, drag_label, mg_vec, mg_label))
        self.wait(1.5)
        
        L_vector = Arrow(start=[0,0,0],end=[-5,0,0], color=BLUE, )
        L_label = MathTex("L")
        
        self.add_fixed_orientation_mobjects(L_label)
        self.move_camera(phi=75*DEGREES,theta=0,gamma=30*DEGREES, added_anims=[GrowArrow(L_vector),
                                                                               L_label.animate.next_to(L_vector, OUT)])
        
        self.move_camera(phi=75*DEGREES,theta=0,gamma=10*DEGREES, added_anims=[ScaleInPlace(L_vector, 0),
                                                                              L_label.animate.scale(.1)])
        self.remove(L_vector)
        self.wait(1.3)
        
        L_vector = Arrow(start=[0,0,0],end=[0,0,3.7], color=BLUE)
        self.begin_ambient_camera_rotation(about="theta", rate = -5)
        self.play(GrowArrow(L_vector), L_label.scale(1).animate.next_to(L_vector, OUT))
        self.wait(3)
        self.play(FadeOut(lift_vec, lift_label, drag_vec, drag_label, mg_vec, mg_label))
        self.begin_ambient_camera_rotation(about="phi", rate=-.3)
        self.wait(4)
        self.stop_ambient_camera_rotation()

KeyboardInterrupt: 